In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy  
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium 
import folium # plotting library
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

!pip install geocoder

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 102kB 4.6MB/s 
Folium installed
Libraries imported.


In [2]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [0]:
import json
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [0]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [0]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [11]:
map_NY = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NY)  
    
map_NY

In [12]:
# @hidden_cell
LIMIT = 500 
radius = 100000 
CLIENT_ID = 'T2TNWRD34JRJGUJB42SHBYPNOU4GPND1PXFYJGHLDBWTXPZL'
CLIENT_SECRET = 'NJL1VDMNB4VMJKIVKX2J4DSSJWEAFM5IS1PPF5TZGVDXUDVZ'
VERSION = '20181020'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T2TNWRD34JRJGUJB42SHBYPNOU4GPND1PXFYJGHLDBWTXPZL
CLIENT_SECRET:NJL1VDMNB4VMJKIVKX2J4DSSJWEAFM5IS1PPF5TZGVDXUDVZ


In [0]:
import urllib
def get_venues(names, latitudes, longitudes, radius=100000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [16]:
college_places = get_venues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], 
                                       longitudes=neighborhoods['Longitude'], radius=5000, categoryIds='4d4b7105d754a06372d81259')
college_places.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,St. Theresa School,40.849301,-73.832999,College Academic Building
1,Wakefield,40.894705,-73.847201,O'Hare Hall,40.859291,-73.882599,College Residence Hall
2,Wakefield,40.894705,-73.847201,Fordham University - Rose Hill,40.861041,-73.885490,University
3,Wakefield,40.894705,-73.847201,Monroe College Jerome Hall & Campus Library,40.863212,-73.900518,General College & University
4,Wakefield,40.894705,-73.847201,Bronxnet,40.871987,-73.896252,College Administrative Building


In [17]:
college_places.shape

(14872, 7)

In [18]:
college_places.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,50,50,50,50,50,50
Annadale,39,39,39,39,39,39
Arden Heights,45,45,45,45,45,45
Arlington,49,49,49,49,49,49
Arrochar,50,50,50,50,50,50
Arverne,16,16,16,16,16,16
Astoria,50,50,50,50,50,50
Astoria Heights,50,50,50,50,50,50
Auburndale,50,50,50,50,50,50


In [19]:
print('There are {} uniques categories.'.format(len(college_places['Venue Category'].unique())))

There are 71 uniques categories.


In [20]:
# one hot encoding
cp_onehot = pd.get_dummies(college_places[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cp_onehot['Neighborhood'] = college_places['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cp_onehot.columns[-1]] + list(cp_onehot.columns[:-1])
cp_onehot = cp_onehot[fixed_columns]

cp_onehot.head()

,Neighborhood,Art Gallery,Athletics & Sports,Auditorium,Bookstore,Café,Church,Coffee Shop,College & University,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Baseball Diamond,College Basketball Court,College Bookstore,College Cafeteria,College Classroom,College Communications Building,College Cricket Pitch,College Engineering Building,College Football Field,College Gym,College Lab,College Library,College Math Building,College Quad,College Rec Center,College Residence Hall,College Science Building,College Soccer Field,College Stadium,College Technology Building,College Theater,College Track,Community College,Deli / Bodega,Dentist's Office,Doctor's Office,Elementary School,Event Space,Food Court,Fountain,Fraternity House,General College & University,Government Building,Gym,Gym / Fitness Center,Gym Pool,High School,Hobby Shop,Hospital,Law School,Library,Martial Arts Dojo,Medical Center,Medical School,Middle School,Music School,Office,Other Great Outdoors,Performing Arts Venue,Sandwich Place,School,Sorority House,Spiritual Center,Student Center,Tennis Court,Track,Trade School,University,Yoga Studio
0,Wakefield,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
cp_onehot.shape

(14872, 72)

In [22]:
cp_grouped=cp_onehot.groupby('Neighborhood').mean().reset_index()
cp_grouped

,Neighborhood,Art Gallery,Athletics & Sports,Auditorium,Bookstore,Café,Church,Coffee Shop,College & University,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Baseball Diamond,College Basketball Court,College Bookstore,College Cafeteria,College Classroom,College Communications Building,College Cricket Pitch,College Engineering Building,College Football Field,College Gym,College Lab,College Library,College Math Building,College Quad,College Rec Center,College Residence Hall,College Science Building,College Soccer Field,College Stadium,College Technology Building,College Theater,College Track,Community College,Deli / Bodega,Dentist's Office,Doctor's Office,Elementary School,Event Space,Food Court,Fountain,Fraternity House,General College & University,Government Building,Gym,Gym / Fitness Center,Gym Pool,High School,Hobby Shop,Hospital,Law School,Library,Martial Arts Dojo,Medical Center,Medical School,Middle School,Music School,Office,Other Great Outdoors,Performing Arts Venue,Sandwich Place,School,Sorority House,Spiritual Center,Student Center,Tennis Court,Track,Trade School,University,Yoga Studio
0,Allerton,0.0,0.00,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.260000,0.100000,0.020000,0.020000,0.000000,0.020000,0.000000,0.040000,0.020000,0.000000,0.0,0.000000,0.020000,0.040000,0.000000,0.060000,0.020000,0.040000,0.00,0.080000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.00,0.00,0.000000,0.0,0.00,0.00,0.000000,0.100000,0.00,0.020000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.0,0.0,0.00,0.040000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.040000,0.00,0.00,0.000000,0.040000,0.00
1,Annadale,0.0,0.00,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.076923,0.000000,0.0,0.000000,0.000000,0.025641,0.025641,0.025641,0.000000,0.000000,0.00,0.025641,0.000000,0.000000,0.000000,0.000000,0.025641,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0,0.00,0.00,0.076923,0.102564,0.00,0.051282,0.000000,0.0,0.000000,0.00,0.00,0.025641,0.0,0.0,0.00,0.025641,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.025641,0.128205,0.000000,0.051282,0.00,0.00,0.051282,0.153846,0.00
2,Arden Heights,0.0,0.00,0.00,0.022222,0.0,0.00,0.000000,0.000000,0.088889,0.000000,0.000000,0.000000,0.000000,0.000000,0.044444,0.000000,0.088889,0.000000,0.0,0.000000,0.000000,0.088889,0.000000,0.000000,0.000000,0.022222,0.00,0.022222,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0,0.00,0.00,0.066667,0.111111,0.00,0.044444,0.000000,0.0,0.000000,0.00,0.00,0.022222,0.0,0.0,0.00,0.022222,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.022222,0.111111,0.000000,0.044444,0.00,0.00,0.044444,0.111111,0.00
3,Arlington,0.0,0.00,0.00,0.020408,0.0,0.00,0.000000,0.040816,0.204082,0.142857,0.020408,0.020408,0.020408,0.000000,0.000000,0.020408,0.000000,0.020408,0.0,0.020408,0.020408,0.020408,0.000000,0.020408,0.020408,0.000000,0.00,0.000000,0.102041,0.020408,0.020408,0.000000,0.000000,0.000000,0.040816,0.00,0.00,0.00,0.000000,0.0,0.00,0.00,0.000000,0.061224,0.00,0.000000,0.020408,0.0,0.000000,0.00,0.00,0.000000,0.0,0.0,0.00,0.020408,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.040816,0.00,0.00,0.020408,0.040816,0.00
4,Arrochar,0.0,0.00,0.00,0.000000,0.0,0.00,0.020000,0.000000,0.060000,0.100000,0.000000,0.020000,0.000000,0.020000,0.020000,0.000000,0.060000,0.020000,0.0,0.000000,0.040000,0.020000,0.000000,0.020000,0.000000,0.040000,0.00,0.080000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0,0.00,0.00,0.020000,0.200000,0.00,0.000000,0.020000,0.0,0.020000,0.00,0.00,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.080000,0.00,0.00,0.040000,0.060000,0.00
5,Arverne,0.0,0.00,0.00,0.000000,0.0,0.00,0.000000,0.125000,0.437500,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [23]:
cp_grouped.shape

(302, 72)

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cp_grouped['Neighborhood']

for ind in np.arange(cp_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cp_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,College Academic Building,College Administrative Building,General College & University,College Residence Hall,College Library,College Quad,College Cafeteria,Medical School,College Gym,Student Center
1,Annadale,University,Sorority House,General College & University,College Classroom,College Academic Building,Fraternity House,Student Center,Gym,Trade School,College Theater
2,Arden Heights,University,General College & University,Sorority House,College Academic Building,College Classroom,College Gym,Fraternity House,College Bookstore,Gym,Trade School
3,Arlington,College Academic Building,College Administrative Building,College Science Building,General College & University,Community College,University,Student Center,College & University,College Cafeteria,Gym / Fitness Center
4,Arrochar,General College & University,College Administrative Building,College Residence Hall,Student Center,University,College Academic Building,College Classroom,College Science Building,College Quad,College Football Field


In [0]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [44]:
# set number of clusters
kclusters = 10

cp_grouped_clustering = cp_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cp_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([6, 0, 0, 1, 3, 4, 6, 9, 1, 8], dtype=int32)

In [45]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,College Academic Building,College Administrative Building,General College & University,College Residence Hall,College Library,College Quad,College Cafeteria,Medical School,College Gym,Student Center
1,Annadale,University,Sorority House,General College & University,College Classroom,College Academic Building,Fraternity House,Student Center,Gym,Trade School,College Theater
2,Arden Heights,University,General College & University,Sorority House,College Academic Building,College Classroom,College Gym,Fraternity House,College Bookstore,Gym,Trade School
3,Arlington,College Academic Building,College Administrative Building,College Science Building,General College & University,Community College,University,Student Center,College & University,College Cafeteria,Gym / Fitness Center
4,Arrochar,General College & University,College Administrative Building,College Residence Hall,Student Center,University,College Academic Building,College Classroom,College Science Building,College Quad,College Football Field


In [46]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

cp_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cp_merged = cp_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

cp_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,6,College Academic Building,General College & University,College Administrative Building,College Residence Hall,College Quad,College Gym,College Cafeteria,College Library,College Classroom,University
1,Bronx,Co-op City,40.874294,-73.829939,6,College Academic Building,College Residence Hall,College Administrative Building,College Cafeteria,College Gym,College Library,Medical School,General College & University,Student Center,University
2,Bronx,Eastchester,40.887556,-73.827806,6,College Academic Building,College Residence Hall,College Gym,General College & University,University,College Cafeteria,College Classroom,Medical School,College Library,College Auditorium
3,Bronx,Fieldston,40.895437,-73.905643,6,College Academic Building,General College & University,College Administrative Building,College Gym,College Library,College Residence Hall,College Quad,Student Center,College Classroom,College Cafeteria
4,Bronx,Riverdale,40.890834,-73.912585,6,College Academic Building,General College & University,College Administrative Building,College Library,College Residence Hall,College Gym,College Quad,Student Center,College Classroom,College Cafeteria


In [47]:
cp_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,6,College Academic Building,General College & University,College Administrative Building,College Residence Hall,College Quad,College Gym,College Cafeteria,College Library,College Classroom,University
1,Bronx,Co-op City,40.874294,-73.829939,6,College Academic Building,College Residence Hall,College Administrative Building,College Cafeteria,College Gym,College Library,Medical School,General College & University,Student Center,University
2,Bronx,Eastchester,40.887556,-73.827806,6,College Academic Building,College Residence Hall,College Gym,General College & University,University,College Cafeteria,College Classroom,Medical School,College Library,College Auditorium
3,Bronx,Fieldston,40.895437,-73.905643,6,College Academic Building,General College & University,College Administrative Building,College Gym,College Library,College Residence Hall,College Quad,Student Center,College Classroom,College Cafeteria
4,Bronx,Riverdale,40.890834,-73.912585,6,College Academic Building,General College & University,College Administrative Building,College Library,College Residence Hall,College Gym,College Quad,Student Center,College Classroom,College Cafeteria
5,Bronx,Kingsbridge,40.881687,-73.902818,6,College Academic Building,General College & University,Medical School,College Residence Hall,College Administrative Building,Student Center,College Library,College Science Building,University,College Classroom
6,Manhattan,Marble Hill,40.876551,-73.910660,6,College Academic Building,General College & University,College Administrative Building,Medical School,College Residence Hall,College Library,College Science Building,Student Center,College Gym,University
7,Bronx,Woodlawn,40.898273,-73.867315,6,College Academic Building,College Administrative Building,General College & University,College Residence Hall,College Quad,College Gym,College Cafeteria,College Library,College Classroom,Student Center
8,Bronx,Norwood,40.877224,-73.879391,6,College Academic Building,General College & University,College Administrative Building,College Residence Hall,College Library,College Cafeteria,College Quad,College Classroom,Student Center,University
9,Bronx,Williamsbridge,40.881039,-73.857446,6,College Academic Building,General College & University,College Administrative Building,College Residence Hall,College Gym,College Quad,College Cafeteria,College Library,College Classroom,College Football Field


In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cp_merged['Latitude'], cp_merged['Longitude'], cp_merged['Neighborhood'], cp_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
cp_merged.loc[cp_merged['Cluster Labels'] == 0, cp_merged.columns[[1] + list(range(5, cp_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
169,Rosedale,College Academic Building,General College & University,College Library,Student Center,College Classroom,Fraternity House,University,College Gym,College Rec Center,College Administrative Building
188,Laurelton,Student Center,College Academic Building,College Library,General College & University,College Gym,College Classroom,College Rec Center,Fraternity House,Trade School,Medical School
193,Brookville,College Academic Building,Student Center,College Library,College Classroom,Fraternity House,General College & University,College Gym,College Rec Center,University,Trade School
213,Great Kills,General College & University,College Classroom,College Academic Building,College Science Building,University,Trade School,Sorority House,College Administrative Building,Fraternity House,College Communications Building
214,Eltingville,General College & University,College Academic Building,College Classroom,College Science Building,University,Trade School,Sorority House,College Administrative Building,College Lab,Fraternity House
215,Annadale,University,Sorority House,General College & University,College Classroom,College Academic Building,Fraternity House,Student Center,Gym,Trade School,College Theater
216,Woodrow,General College & University,Fraternity House,Sorority House,University,College Classroom,College Academic Building,College Bookstore,Trade School,College Gym,Gym
217,Tottenville,Trade School,College Classroom,College Gym,University,Sorority House,Student Center,Fraternity House,College Library,General College & University,College Bookstore
236,Huguenot,University,Fraternity House,Sorority House,College Academic Building,General College & University,College Classroom,Student Center,Gym,Trade School,College Gym
237,Pleasant Plains,Trade School,College Classroom,Sorority House,Fraternity House,University,College Academic Building,Student Center,Gym,College Gym,General College & University


In [33]:
cp_merged.loc[cp_merged['Cluster Labels'] == 1, cp_merged.columns[[1] + list(range(5, cp_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Bay Ridge,General College & University,College Academic Building,Student Center,College Classroom,College Residence Hall,College Administrative Building,Elementary School,College Quad,University,School
71,Cypress Hills,General College & University,College Academic Building,College Classroom,Student Center,College Administrative Building,Trade School,College Auditorium,High School,College Arts Building,College Rec Center
72,East New York,College Academic Building,General College & University,College Administrative Building,Student Center,College Classroom,College Science Building,Medical School,College Library,University,School
81,Dyker Heights,General College & University,Student Center,College Academic Building,College Classroom,College Residence Hall,College Administrative Building,University,Elementary School,Middle School,College Bookstore
85,Sea Gate,General College & University,College Classroom,Student Center,College Academic Building,College Bookstore,College Arts Building,Elementary School,College Communications Building,College Track,Community College
99,Fort Hamilton,General College & University,College Academic Building,Student Center,College Administrative Building,College Residence Hall,College Classroom,University,Elementary School,College Quad,College Arts Building
155,Glen Oaks,General College & University,College Residence Hall,Medical School,College Academic Building,University,College Library,College Arts Building,College Administrative Building,College Science Building,College Lab
168,Cambria Heights,College Administrative Building,Student Center,College Cafeteria,General College & University,College Residence Hall,College Gym,College Classroom,College Academic Building,College Bookstore,University
181,Floral Park,General College & University,College Academic Building,College Residence Hall,University,Student Center,College Science Building,College Lab,College Gym,Medical School,College Arts Building
197,St. George,General College & University,College Academic Building,Student Center,College Administrative Building,College Residence Hall,University,College Quad,College Football Field,College Gym,College Science Building


In [34]:
cp_merged.loc[cp_merged['Cluster Labels'] == 2, cp_merged.columns[[1] + list(range(5, cp_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,High Bridge,College Academic Building,College Library,College Residence Hall,Student Center,General College & University,College Arts Building,College Administrative Building,College Cafeteria,College Quad,College Science Building
20,Melrose,College Academic Building,College Library,College Residence Hall,Student Center,General College & University,College Arts Building,College Administrative Building,College Cafeteria,College Quad,College Science Building
21,Mott Haven,College Academic Building,College Library,Student Center,College Residence Hall,General College & University,College Science Building,College Quad,College Cafeteria,College Administrative Building,College Arts Building
22,Port Morris,College Academic Building,College Library,College Residence Hall,Student Center,College Administrative Building,College Arts Building,Hospital,College Quad,College Cafeteria,Deli / Bodega
43,Concourse,College Academic Building,College Library,College Residence Hall,Student Center,General College & University,College Arts Building,College Administrative Building,College Cafeteria,College Quad,College Science Building
49,Greenpoint,College Academic Building,College Administrative Building,College Library,Law School,University,College Auditorium,Hospital,College Theater,Student Center,College Technology Building
61,Williamsburg,College Academic Building,College Administrative Building,College Library,Hospital,University,College Auditorium,General College & University,Law School,Student Center,College Theater
64,Brooklyn Heights,College Academic Building,College Library,College Theater,College Administrative Building,General College & University,Student Center,University,College Auditorium,College Math Building,Government Building
65,Cobble Hill,College Academic Building,College Library,General College & University,Law School,College Classroom,College Auditorium,College Administrative Building,College Theater,Student Center,College Cafeteria
66,Carroll Gardens,College Academic Building,College Library,General College & University,Law School,College Classroom,College Auditorium,College Administrative Building,College Theater,Student Center,College Cafeteria


In [35]:
cp_merged.loc[cp_merged['Cluster Labels'] == 3, cp_merged.columns[[1] + list(range(5, cp_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,College Academic Building,General College & University,College Administrative Building,College Residence Hall,College Quad,College Gym,College Cafeteria,College Library,College Classroom,University
1,Co-op City,College Academic Building,College Residence Hall,College Administrative Building,College Cafeteria,College Gym,College Library,Medical School,General College & University,Student Center,University
2,Eastchester,College Academic Building,College Residence Hall,College Gym,General College & University,University,College Cafeteria,College Classroom,Medical School,College Library,College Auditorium
3,Fieldston,College Academic Building,General College & University,College Administrative Building,College Gym,College Library,College Residence Hall,College Quad,Student Center,College Classroom,College Cafeteria
4,Riverdale,College Academic Building,General College & University,College Administrative Building,College Library,College Residence Hall,College Gym,College Quad,Student Center,College Classroom,College Cafeteria
5,Kingsbridge,College Academic Building,General College & University,Medical School,College Residence Hall,College Administrative Building,Student Center,College Library,College Science Building,University,College Classroom
6,Marble Hill,College Academic Building,General College & University,College Administrative Building,Medical School,College Residence Hall,College Library,College Science Building,Student Center,College Gym,University
7,Woodlawn,College Academic Building,College Administrative Building,General College & University,College Residence Hall,College Quad,College Gym,College Cafeteria,College Library,College Classroom,Student Center
8,Norwood,College Academic Building,General College & University,College Administrative Building,College Residence Hall,College Library,College Cafeteria,College Quad,College Classroom,Student Center,University
9,Williamsbridge,College Academic Building,General College & University,College Administrative Building,College Residence Hall,College Gym,College Quad,College Cafeteria,College Library,College Classroom,College Football Field


In [36]:
cp_merged.loc[cp_merged['Cluster Labels'] == 4, cp_merged.columns[[1] + list(range(5, cp_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,Schuylerville,College Academic Building,College Classroom,College Residence Hall,Student Center,College Rec Center,Fraternity House,College Administrative Building,Medical School,University,High School
40,Castle Hill,College Academic Building,College Classroom,Medical School,College Residence Hall,University,College Rec Center,College Administrative Building,Trade School,High School,Student Center
47,Bensonhurst,College Academic Building,General College & University,College Classroom,College Quad,University,Student Center,Elementary School,College Arts Building,College Cafeteria,College Bookstore
48,Sunset Park,College Academic Building,General College & University,College Classroom,College Library,College Rec Center,College Quad,Elementary School,College Cafeteria,Medical School,College Administrative Building
50,Gravesend,College Academic Building,General College & University,College Classroom,College Arts Building,College Cafeteria,College Math Building,College Quad,Elementary School,College Bookstore,College Library
51,Brighton Beach,College Academic Building,College Classroom,General College & University,College Math Building,College Arts Building,College Quad,College Cafeteria,College Library,College Science Building,Medical School
52,Sheepshead Bay,College Academic Building,College Classroom,General College & University,College Arts Building,College Library,College Cafeteria,College Math Building,School,College Quad,High School
53,Manhattan Terrace,College Academic Building,General College & University,College Classroom,College Library,College Cafeteria,Elementary School,Medical School,College Math Building,University,Student Center
54,Flatbush,College Academic Building,College Classroom,General College & University,College Library,Elementary School,College Gym,College Lab,Medical School,College Quad,College Cafeteria
55,Crown Heights,College Academic Building,College Library,College Classroom,General College & University,College Lab,College Cafeteria,College Quad,College Administrative Building,Medical School,Law School


In [49]:
cp_merged.loc[cp_merged['Cluster Labels'] == 5, cp_merged.columns[[1] + list(range(5, cp_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Sunset Park,College Academic Building,General College & University,College Classroom,College Library,College Rec Center,College Quad,Elementary School,College Cafeteria,Medical School,College Administrative Building
54,Flatbush,College Academic Building,College Classroom,General College & University,College Library,Elementary School,College Gym,College Lab,Medical School,College Quad,College Cafeteria
55,Crown Heights,College Academic Building,College Library,College Classroom,General College & University,College Lab,College Cafeteria,College Quad,College Administrative Building,Medical School,Law School
56,East Flatbush,College Academic Building,College Classroom,General College & University,College Library,College Quad,College Gym,College Lab,Medical School,College Cafeteria,College Science Building
57,Kensington,College Academic Building,College Library,College Classroom,General College & University,College Lab,College Quad,College Cafeteria,College Gym,Medical School,Elementary School
58,Windsor Terrace,College Academic Building,College Classroom,College Library,General College & University,College Quad,College Lab,College Cafeteria,College Administrative Building,College Arts Building,College Theater
59,Prospect Heights,College Academic Building,General College & University,College Classroom,College Lab,College Administrative Building,College Quad,College Library,College Arts Building,College Theater,College Cafeteria
60,Brownsville,College Academic Building,College Classroom,General College & University,College Library,University,College Quad,College Gym,College Lab,Medical School,College Cafeteria
62,Bushwick,College Academic Building,College Library,College Classroom,College Administrative Building,General College & University,College Cafeteria,College Lab,Student Center,College Gym,University
63,Bedford Stuyvesant,College Academic Building,College Classroom,College Library,General College & University,College Administrative Building,Law School,College Cafeteria,College Quad,College Lab,College Arts Building


In [50]:
cp_merged.loc[cp_merged['Cluster Labels'] == 6, cp_merged.columns[[1] + list(range(5, cp_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,College Academic Building,General College & University,College Administrative Building,College Residence Hall,College Quad,College Gym,College Cafeteria,College Library,College Classroom,University
1,Co-op City,College Academic Building,College Residence Hall,College Administrative Building,College Cafeteria,College Gym,College Library,Medical School,General College & University,Student Center,University
2,Eastchester,College Academic Building,College Residence Hall,College Gym,General College & University,University,College Cafeteria,College Classroom,Medical School,College Library,College Auditorium
3,Fieldston,College Academic Building,General College & University,College Administrative Building,College Gym,College Library,College Residence Hall,College Quad,Student Center,College Classroom,College Cafeteria
4,Riverdale,College Academic Building,General College & University,College Administrative Building,College Library,College Residence Hall,College Gym,College Quad,Student Center,College Classroom,College Cafeteria
5,Kingsbridge,College Academic Building,General College & University,Medical School,College Residence Hall,College Administrative Building,Student Center,College Library,College Science Building,University,College Classroom
6,Marble Hill,College Academic Building,General College & University,College Administrative Building,Medical School,College Residence Hall,College Library,College Science Building,Student Center,College Gym,University
7,Woodlawn,College Academic Building,College Administrative Building,General College & University,College Residence Hall,College Quad,College Gym,College Cafeteria,College Library,College Classroom,Student Center
8,Norwood,College Academic Building,General College & University,College Administrative Building,College Residence Hall,College Library,College Cafeteria,College Quad,College Classroom,Student Center,University
9,Williamsbridge,College Academic Building,General College & University,College Administrative Building,College Residence Hall,College Gym,College Quad,College Cafeteria,College Library,College Classroom,College Football Field


In [51]:
cp_merged.loc[cp_merged['Cluster Labels'] == 7, cp_merged.columns[[1] + list(range(5, cp_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
191,Rockaway Park,College Academic Building,College Communications Building,College Engineering Building,College Football Field,College Gym,College Lab,College Library,College Math Building,College Quad,Yoga Studio


In [52]:
cp_merged.loc[cp_merged['Cluster Labels'] == 8, cp_merged.columns[[1] + list(range(5, cp_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Bensonhurst,College Academic Building,General College & University,College Classroom,College Quad,University,Student Center,Elementary School,College Arts Building,College Cafeteria,College Bookstore
50,Gravesend,College Academic Building,General College & University,College Classroom,College Arts Building,College Cafeteria,College Math Building,College Quad,Elementary School,College Bookstore,College Library
51,Brighton Beach,College Academic Building,College Classroom,General College & University,College Math Building,College Arts Building,College Quad,College Cafeteria,College Library,College Science Building,Medical School
52,Sheepshead Bay,College Academic Building,College Classroom,General College & University,College Arts Building,College Library,College Cafeteria,College Math Building,School,College Quad,High School
53,Manhattan Terrace,College Academic Building,General College & University,College Classroom,College Library,College Cafeteria,Elementary School,Medical School,College Math Building,University,Student Center
74,Canarsie,College Academic Building,General College & University,College Cafeteria,College Science Building,College Classroom,College Library,College Quad,High School,Medical School,College Arts Building
75,Flatlands,College Academic Building,General College & University,College Classroom,College Library,College Cafeteria,High School,University,Student Center,Medical School,College Math Building
76,Mill Island,College Academic Building,General College & University,College Classroom,College Arts Building,University,College Library,College Cafeteria,School,College Math Building,College Quad
77,Manhattan Beach,College Academic Building,College Classroom,General College & University,College Arts Building,College Math Building,College Quad,College Library,College Cafeteria,University,Elementary School
78,Coney Island,College Academic Building,General College & University,College Classroom,College Bookstore,College Math Building,Trade School,College Gym,College Lab,College Library,Middle School


In [53]:
cp_merged.loc[cp_merged['Cluster Labels'] == 9, cp_merged.columns[[1] + list(range(5, cp_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,East New York,College Academic Building,General College & University,College Administrative Building,Student Center,College Classroom,College Science Building,Medical School,College Library,University,School
73,Starrett City,College Academic Building,Student Center,General College & University,College Administrative Building,College Science Building,College Library,College Classroom,Medical School,College Cafeteria,School
140,Sunnyside,College Academic Building,College Administrative Building,General College & University,College Library,College Science Building,University,Student Center,College Residence Hall,College Communications Building,College Cafeteria
155,Glen Oaks,General College & University,College Residence Hall,Medical School,College Academic Building,University,College Library,College Arts Building,College Administrative Building,College Science Building,College Lab
156,Bellerose,General College & University,College Administrative Building,College Academic Building,College Residence Hall,College Cafeteria,Medical School,College Library,University,College Bookstore,College Quad
168,Cambria Heights,College Administrative Building,Student Center,College Cafeteria,General College & University,College Residence Hall,College Gym,College Classroom,College Academic Building,College Bookstore,University
181,Floral Park,General College & University,College Academic Building,College Residence Hall,University,Student Center,College Science Building,College Lab,College Gym,Medical School,College Arts Building
201,West Brighton,College Academic Building,College Administrative Building,General College & University,Student Center,University,College Residence Hall,College Football Field,College Science Building,College Quad,College Classroom
203,Todt Hill,College Administrative Building,General College & University,College Science Building,College Academic Building,Student Center,University,College Residence Hall,College Library,College Quad,College Basketball Court
205,Port Richmond,College Academic Building,College Administrative Building,College Science Building,General College & University,Student Center,University,College Residence Hall,College Library,College Quad,College Baseball Diamond
